In [1]:
import torch
import train_methods as tm
from tqdm import tqdm
from UNet import Unet
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# global var
C = 3                  # number of channels to give the model
batch_size = 16
epochs = 5
learning_rate = 1e-2
# act_batch_size = 1
momentum = 0.99
device = "cuda" if torch.cuda.is_available() else "cpu"
channels = [C, 64, 128, 256, 512, 1024]
w0 = 10
sigma = 5
model_path = './model.pt'
Train_image_dir = 'amos22/Train/input'
Train_label_dir = 'amos22/Train/label'
Val_image_dir = 'amos22/Validation/input'
Val_label_dir = 'amos22/Validation/label'
TEMP_FOLSER = "TEMP"

/home/or/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# Early stopping
es = tm.EarlyStopping(patience=100, fname=model_path)


# debug
# imagesTR_len = len(os.listdir("/amos22/imagesTr"))
# print(f"Number of files in imagesTr: {imagesTR_len}")


# Make progress bars
# pbar_epoch = tqdm(total=epochs, unit='epoch', position=0, leave=False)
# pbar_train = tqdm(total=imagesTR_len, unit='batch', position=1, leave=False)

# Make model
model = Unet(channels=channels, no_classes=1).double().to(device)
model = model.to(device).float()


# Make adam optimiser
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,  # Learning rate
    betas=(0.9, 0.999),  # Coefficients used for computing running averages of gradient and its square
    eps=1e-08,  # Term added to denominator to improve numerical stability
    weight_decay=0  # Weight decay (L2 penalty)
)

# Make loss
criterion = torch.nn.CrossEntropyLoss()

In [4]:
# Load checkpoint (if it exists)
cur_epoch = 0
if os.path.isfile(model_path):
    checkpoint = torch.load(model_path)
    cur_epoch = checkpoint['epoch']
    es.best_loss = checkpoint['loss']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Hold stats for training process
stats = {'epoch': [], 'train_loss': [], 'val_loss': []}

# Training  / validation loop
for epoch in range(cur_epoch, epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    # Train / validate
    # pbar_epoch.set_description_str(f'Epoch {epoch + 1}')
    train_loss = tm.train(model, optimizer, batch_size, TEMP_FOLSER, TEMP_FOLSER, criterion, device)
    print(f"Train loss: {train_loss}")
    print(f"end of train epoch{epoch}")
    val_loss = tm.validate(model, batch_size, TEMP_FOLSER,TEMP_FOLSER, criterion, device)
    print(f"end of val epoch{epoch}")


    # Append stats
    stats['epoch'].append(epoch)
    stats['train_loss'].append(train_loss)
    stats['val_loss'].append(val_loss)

    # Early stopping (just saves model if validation loss decreases when: pass)
    if es(epoch, val_loss, optimizer, model): pass


Epoch 1/5
befoer encoding
after encoding and before decoding
after decoding
after segmentation


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4

In [ ]:
# load model

model      = Unet(channels = channels, no_classes = 1).double().to(device)
checkpoint = torch.load(model_path)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Make loss
criterion = torch.CrossEntropyLoss()
with torch.no_grad():

    for  (X, y) in enumerate(test_loader):

        # Forward
        y_hat = model(X)
        y_hat = torch.sigmoid(y_hat)


        # Convert to numpy
        X = np.squeeze(X.cpu().numpy())
        y = np.squeeze(y.cpu().numpy())
        y_hat = np.squeeze(y_hat.detach().cpu().numpy())

        # Make mask
        y_hat2 = y_hat > 0.5

        # plot
        fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 8))

        ax[0].imshow(y, 'gray', interpolation = None)
        ax[0].axis('off');
        ax[0].set_title('Target');

        ax[1].imshow(y_hat, 'gray', interpolation = None)
        ax[1].axis('off');
        ax[1].set_title('Prediction');